<a href="https://colab.research.google.com/github/CupOfDim/neuron/blob/main/Create_Neuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

class Neuron():
  def __init__(self, weight, bias):
    self.weight = weight
    self.bias = bias
  def foreward(self, inputs):
    total = np.dot(self.weight, inputs) + self.bias
    return sigmoid(total)

In [ ]:
weight = np.array([0,1])
bias = 4
n = Neuron(weight, bias)
x = np.array([2,3])
print(n.foreward(x))

0.9990889488055994


In [ ]:
class NeuronNet():
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

    self.h1 = Neuron(self.weights, self.bias)
    self.h2 = Neuron(self.weights, self.bias)
    self.o1 = Neuron(self.weights, self.bias)
  
  def foreward(self,x):
    out_h1 = self.h1.foreward(x)
    out_h2 = self.h2.foreward(x)

    out_o1 = self.o1.foreward(np.array([out_h1, out_h2]))
    return out_o1

In [ ]:
weights = [0,1]
bias = 1
n_net = NeuronNet(weights, bias)
x = [2,3]
print(n_net.foreward(np.array(x)))

0.8788956696366197


In [ ]:
df = pd.DataFrame(columns=["Weight", "Height", "Gender"])

In [ ]:
df["Weight"] = np.array([133, 160, 152, 120])
df["Height"] = np.array([65, 72, 70, 60])
df["Gender"] = np.array([1,0,0,1])
df.head()

In [ ]:
y = df["Gender"]
df = df.drop("Gender", axis=1)


In [ ]:
sum(df["Weight"])/len(df["Weight"])
for col in np.array(df.columns):
  df[col] -= sum(df[col])/len(df[col])
df

In [ ]:
def mse_loss(y_true, y_pred):
  return ((y_true - y_pred)**2).mean()


In [ ]:
def deriv_sigmoid(x):
  fx = sigmoid(x)
  return fx * (1 - fx)

In [ ]:
class RecogGenderNeuron():
  def __init__(self):
        # Вес
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()
 
        # Смещения
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()
 
  def feedforward(self, x):
        # x является массивом numpy с двумя элементами
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1
  def train(self, data, all_y, learn_rate=0.1, epoches=1000):
    for epoch in range(epoches):
      for x, y in zip(data, all_y):
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        d_L_d_ypred = -2 * (y - y_pred)

        # Нейрон o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Нейрон h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Нейрон h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Обновляем вес и смещения
        # Нейрон h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Нейрон h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Нейрон o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

    # --- Подсчитываем общую потерю в конце каждой фазы
      if epoch % 10 == 0:
          y_preds = np.apply_along_axis(self.feedforward, 1, data)
          loss = mse_loss(all_y, y_preds)
          print("Epoch %d loss: %.3f" % (epoch, loss))

In [ ]:
data = np.array([
    [-2, -1],    # Alice
    [25, 6],     # Bob
    [17, 4],     # Charlie
    [-15, -6], # Diana
])
 
y = np.array([
    1, # Alice
    0, # Bob
    0, # Charlie
    1, # Diana
])

network = RecogGenderNeuron()
network.train(data, y)


Epoch 0 loss: 0.280
Epoch 10 loss: 0.153
Epoch 20 loss: 0.092
Epoch 30 loss: 0.066
Epoch 40 loss: 0.051
Epoch 50 loss: 0.042
Epoch 60 loss: 0.035
Epoch 70 loss: 0.030
Epoch 80 loss: 0.026
Epoch 90 loss: 0.023
Epoch 100 loss: 0.020
Epoch 110 loss: 0.018
Epoch 120 loss: 0.016
Epoch 130 loss: 0.015
Epoch 140 loss: 0.014
Epoch 150 loss: 0.013
Epoch 160 loss: 0.012
Epoch 170 loss: 0.011
Epoch 180 loss: 0.010
Epoch 190 loss: 0.010
Epoch 200 loss: 0.009
Epoch 210 loss: 0.009
Epoch 220 loss: 0.008
Epoch 230 loss: 0.008
Epoch 240 loss: 0.007
Epoch 250 loss: 0.007
Epoch 260 loss: 0.007
Epoch 270 loss: 0.006
Epoch 280 loss: 0.006
Epoch 290 loss: 0.006
Epoch 300 loss: 0.006
Epoch 310 loss: 0.006
Epoch 320 loss: 0.005
Epoch 330 loss: 0.005
Epoch 340 loss: 0.005
Epoch 350 loss: 0.005
Epoch 360 loss: 0.005
Epoch 370 loss: 0.005
Epoch 380 loss: 0.004
Epoch 390 loss: 0.004
Epoch 400 loss: 0.004
Epoch 410 loss: 0.004
Epoch 420 loss: 0.004
Epoch 430 loss: 0.004
Epoch 440 loss: 0.004
Epoch 450 loss: 0.004